In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
class NeuralNetwork:

    def train(self, dataframe, neurons_in_first_layer = 4, test_size = 0.2,iterations = 20000,learning_rate = 0.01):

        self.df = dataframe
        self.neurons_in_first_layer = neurons_in_first_layer
        
        self.x = self.df.iloc[:, :-1]
        self.y = self.df.iloc[:,-1]

        # Spliting data into training set and test set
        self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(self.x, self.y, test_size=test_size, random_state=42)

        # converting x_train from dataframe format to numpy format for easier usage.
        self.x_train = self.x_train.to_numpy()
        self.y_train = self.y_train.to_numpy()

        # Providing random values for Weights and Biases between -5 and 5
        self.initialize_weights()

        # Starting gradient decent
        iter = 0

        tolerance = 1e-3

        while not (not ((np.linalg.norm(self.dW1()) > tolerance) and (np.linalg.norm(self.dW2()) > tolerance)) or not iter< iterations):
            self.gradient_descent(learning_rate=learning_rate)
            iter += 1
            # print(f"\n\niter = {iter}")
            # print(f"self.dW1 = {self.dW1()}")
            # print(f"self.dW2 = {self.dW2()}")
        print(f"\n\niter = {iter}")
        print(f"self.dW1 = {self.dW1()}")
        print(f"self.dW2 = {self.dW2()}")
        

    def initialize_weights(self):
        # generating initial guess for W matrix
        np.random.seed(seed=6)
        self.W1 = 10 * np.random.rand(self.neurons_in_first_layer,self.x_train.shape[1]) - 5
        self.W2 = 10 * np.random.rand(1,self.neurons_in_first_layer) - 5

        # generating initial guess for b matrix
        self.b1 = 10 * np.random.rand(self.neurons_in_first_layer,1) - 5
        self.b2 = 10 * np.random.rand(1,1) - 5

    def tanh(self,x):
        return np.tanh(x)
    
    def d_tanh(self,x):
        return 1 - np.square(np.tanh(x))
    
    def logloss(self,y, a):
        return -(y*np.log(a) + (1-y)*np.log(1-a))

    def d_logloss(self,y, a):
        return (a - y)/(a*(1 - a))

    def gradient_descent(self,learning_rate):
        # Updating the value of the weights
        self.W1 = self.W1 - (learning_rate * self.dW1())
        self.W2 = self.W2 - (learning_rate * self.dW2())

        # Updating the value of the bias
        self.b1 = self.b1 - (learning_rate * self.db1())
        self.b2 = self.b2 - (learning_rate * self.db2())


    def Z1(self):
        return np.dot(self.W1,self.x_train.T) + self.b1
    
    def Z2(self):
        return np.dot(self.W2,self.a1()) + self.b2

    def Z(self,W,x_T,b):
        '''
        z = (W * x) + b
        Only initially, when input parameters are passed as x_train,
        we have to transpose it while giving it in the function,
        like as follows,
        -> z(W = W1 , x_T = x_train.T , b = b1)

        Else for hidden layers,
        pass x_T normally with giving any transpose to a1,
        like as follows,
        -> z(W = W2 , x_T = a1 , b = b2)

        THERE IS NO NEED FOR CONVERT b FROM nx1 TO nxm,
        CODE ->
        a = np.array([[1,2,3,4],
             [5,6,7,8],
             [9,10,11,12],])
        b = np.array([[5],
             [6],
             [7],])

        print(a+b)  

        OUTPUT ->
        [[ 6  7  8  9]
        [11 12 13 14]
        [16 17 18 19]]

        '''
        return np.dot(W,x_T) + b


    def a(self,z):
        return self.tanh(z)  

    def a1(self):
        return self.tanh(self.Z1()) 
    
    def a2(self):
        return self.tanh(self.Z2())
    
    def dZ2(self):
        '''
        Remember that, here, y_train is a 1xm matrix, not a mx1 matrix,
        so dont forget to convert self.y_train from mx1 to 1xm.
        '''
        dA = self.d_logloss(self.y_train,self.a2())
        dZ2 = np.multiply(self.d_tanh(self.Z2()), dA)
        return dZ2
    
    def dW2(self):
        # print(self.x_train.shape)
        dW2 = 1/self.dZ2().shape[1] * np.dot(self.dZ2(), self.a1().T)
        return dW2
    
    def db2(self):
        db2 = 1/self.dZ2().shape[1] * np.sum(self.dZ2(), axis=1, keepdims=True)
        return db2
    
    def dZ1(self):
        dA = np.dot(self.W2.T, self.dZ2())
        dZ2 = np.multiply(self.d_tanh(self.Z1()), dA)
        return dZ2

    
    def dW1(self):
        dW1 = 1/self.dZ1().shape[1] * np.dot(self.dZ1(), self.x_train)
        return dW1
    
    def db1(self):
        db1 = 1/self.dZ1().shape[1] * np.sum(self.dZ1(), axis=1, keepdims=True)
        return db1
    
    def test_model(self):
        Z1 = self.Z(b=self.b1,W=self.W1,x_T=self.x_test.T,)
        a1 = self.a(z=Z1)
        Z2 = self.Z(b=self.b2,W=self.W2,x_T=a1)
        a2 = self.a(z=Z2)

        self.predicted_correct = 0
        self.predicted_wrong = 0
        self.true_positives = 0
        self.true_negatives = 0
        self.false_positives = 0
        self.false_negatives = 0

        for i,j in zip(self.y_test,a2[0]):
            # print(f"i = {i}  j = {j}")
            if j>0:
                j=1
            else:
                j=0
            
            if i == j:
                self.predicted_correct+=1
                if i == 1:
                    self.true_positives+=1
                else:
                    self.true_negatives+=1
            else:
                self.predicted_wrong+=1
                if j == 1:
                    self.false_positives+=1
                else:
                    self.false_negatives+=1

        print(f"The accuracy of the model is {(self.predicted_correct/(self.predicted_correct + self.predicted_wrong))*100} %.")
        try:
            self.precision = (self.true_positives/(self.true_positives + self.false_positives))
            print(f"The Precision of the model is {self.precision*100} %.")
        except:
            print(f"Precision can't be calculated.")
        
        try:
            self.recall = (self.true_positives/(self.true_positives + self.false_negatives))
            print(f"The Recall/Sensitivity of the model is {self.recall*100} %.")
        except:
            print(f"Prediction can't be calculated.")

        try:
            self.specificity = (self.true_negatives/(self.true_negatives + self.false_positives))
            print(f"The Specificity of the model is {self.specificity*100} %.")
        except:
            print(f"Specificity can't be calculated.")

        try:
            self.f1_score = 2 * ((self.precision * self.recall)/(self.precision + self.recall))
            print(f"The F1-Score of the model is {self.f1_score}")
        except:
            print(f"F1-Score can't be calculated.")
    
    def cost(self):
        print(self.a2())
        loss = -np.mean(self.y_train * np.log(self.a2().T) + (1 - self.y_train) * np.log(1 - self.a2().T))
        print(loss)
    


In [3]:
df = pd.read_csv("Logistic_regression_ls.csv")
neu1 = NeuralNetwork()
neu1.train(dataframe=df,neurons_in_first_layer=3,test_size=0.2,iterations=10000,learning_rate=0.01)



iter = 9837
self.dW1 = [[ 7.05875539e-07  2.15142089e-06]
 [ 2.84922440e-04  1.44449206e-04]
 [-3.70408771e-04  8.72079178e-04]]
self.dW2 = [[-0.00090105 -0.00247379  0.00638234]]


In [4]:
neu1.test_model()

The accuracy of the model is 100.0 %.
The Precision of the model is 100.0 %.
The Recall/Sensitivity of the model is 100.0 %.
The Specificity of the model is 100.0 %.
The F1-Score of the model is 1.0
